# Итоговые новости акций

In [1]:
# Библиотеки
import pandas as pd
import numpy as np

import requests as rq
import urllib.request
from urllib.parse import urlencode
from urllib.request import urlopen

from bs4 import BeautifulSoup as bs
import json
import re
import io
import os
import time

import datetime as dt

import datetime
from datetime import datetime, date, timedelta

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
stock = ['SBER', 'GAZP', 'ROSN', 'RTKM', 'MOEX']

In [3]:
l = ["сберб|sberb", "rosneft|роснефт", "gazprom|газпром", "rostelecom|ростелеком", "moex|мосбирж"]

In [4]:
yesterday = date.today() - timedelta(days=1)

## Интерфакс

In [5]:
def date_range(start, end, intv):
    start = datetime.strptime(start,"%Y%m%d")
    end = datetime.strptime(end,"%Y%m%d")
    diff = (end  - start ) / intv
    for i in range(intv):
        yield (start + diff * i).strftime("%Y/%m/%d")
    yield end.strftime("%Y/%m/%d")

In [6]:
def parse_interfax_news(days=0):
   
    ress = []
    
    if days == 0:
        data_list = [str(date.today()).replace("-", "/")]
    else:
        data_end = str(date.today()).replace("-", "")
        data_start = "{:%Y%m%d}".format(date.today() - timedelta(days=days))
        data_list= list(date_range(data_start, data_end, days))
    
    CLEANR = re.compile('<.*?>') 
    
    
    for url in data_list:
        interfax_business_url = 'https://www.interfax.ru/news/' + url
        n = rq.get(interfax_business_url).content
        soup = bs(n, 'html.parser')
        n = soup.body.find('div', {'class':'pages'})
        news_urls = re.findall('/news/'+ url+'/all/\w+', str(n))
        news_urls = list(map(lambda x: 'https://www.interfax.ru' + x, news_urls))
        news_urls.insert(0,interfax_business_url ) 
        
        
        for ur in news_urls:
                    response = rq.get(ur)
                    news_u = re.findall('/business/\w+', response.text)
                    news_u = list(map(lambda x: 'https://www.interfax.ru' + x, news_u))
                    
                    
                    for u in news_u:
                                res = {}
                                nv = rq.get(u).content
                                s = bs(nv, 'html.parser')
                                res['time'] = s.find('meta', {'itemprop': 'datePublished'})
                                res['time'] = str(re.findall('\d{4}-\d+-\d+T\d+:\d+:\d+', str(res['time']))[0]).replace("T", " ")
                                res['url']= u
                                res['site'] = 'interfax_business'
                                res['title'] = s.body.find(itemprop="headline").text
                                res['text'] = s.body.find('article', {"itemprop":"articleBody"}).find_all('p')
                                res['text'][0] = ' '.join(str(res['text'][0]).split(' ')[5:])
                                res['text'] = re.sub(CLEANR, '', str(res['text'])).replace("\'", "").replace("]", "").replace("[", "").replace(".,", ".")
                                ress.append(res)   
     
    
    ress = [i for j, i in enumerate(ress) if i['time'][:10].replace("-", "/") in data_list]

    
    return  pd.json_normalize(ress)

In [7]:
df1 = pd.read_csv('interfax_business_new.csv')
df1 = df1.sort_values(by=['time'], ascending=False)
t = (datetime.today().date() - datetime.strptime(df1[['time']].values[0][0][:10], '%Y-%m-%d').date()).days
df2 = parse_interfax_news(t)
day = pd.concat([df1, df2], axis=0, ignore_index=True)
day = day.drop_duplicates (subset=['time'], keep='last')
day = day.sort_values(by=['time'], ascending=True)
day = day[day['time'].map(lambda x: x[0:10]) <= str(yesterday)]
day.to_csv('interfax_business_new.csv', index = False)

In [8]:
inter = pd.read_csv('interfax_business_new.csv')
inter = inter.dropna()
inter['stock'] = ''
for i, j in list(zip(stock,l)):
      inter['stock'][inter['text'].str.lower().str.contains(j)] = i

result = pd.concat([inter[inter['stock'] == 'SBER'], inter[inter['stock'] == 'ROSN'], inter[inter['stock'] == 'GAZP'], inter[inter['stock'] == 'RTKM'], inter[inter['stock'] == 'MOEX']], axis=0, ignore_index=True)        
result

,time,url,site,title,text,stock
0,2022-02-01 05:21:00,https://www.interfax.ru/business/819490,interfax_business,Сбербанк поднял ставки по ипотеке на 1 и более...,Сбербанк поднял ставки по рыночным ипотечным п...,SBER
1,2022-02-01 11:32:00,https://www.interfax.ru/business/819543,interfax_business,Сбербанк в партнерстве с Alipay запустил перев...,Сбербанк запустил денежные переводы в Китай по...,SBER
2,2022-02-01 17:18:00,https://www.interfax.ru/business/819635,interfax_business,"""Золотая корона"" запретила транзакции с крипто...",Система денежных переводов KoronaPay ввела зап...,SBER
3,2022-03-01 00:35:00,https://www.interfax.ru/business/825316,interfax_business,Сбербанк резко повысил ставки по ипотеке вслед...,Сбербанк вслед за резким увеличением ключевой ...,SBER
4,2022-03-01 08:30:00,https://www.interfax.ru/business/825346,interfax_business,"S&P понизило рейтинги ""СОГАЗа"", ""Ингосстраха"" ...",Агентство S&amp;P Global Ratings понизило рейт...,SBER
...,...,...,...,...,...,...
327,2023-11-13 18:58:00,https://www.interfax.ru/business/930340,interfax_business,Рынок акций РФ начал неделю с умеренного роста...,Рынок акций РФ в понедельник подрос по индекса...,MOEX
328,2023-11-14 18:50:00,https://www.interfax.ru/business/930513,interfax_business,Мосбиржа запустила недельные опционы на природ...,"Участникам торгов на срочном рынке ""Московской...",MOEX
329,2023-11-14 18:59:00,https://www.interfax.ru/business/930515,interfax_business,Рынок акций РФ во вторник снизился по большинс...,Рынок акций РФ во вторник снизился по большинс...,MOEX
330,2023-11-15 10:20:00,https://www.interfax.ru/business/930570,interfax_business,Рынок акций РФ открылся снижением индексов РТС...,Рынок акций РФ открылся в среду снижением рубл...,MOEX


## РБК

In [9]:
RBC_FINANCE_URL = 'https://rbc.ru/v10/ajax/get-news-by-filters/?category=finances&limit=20&offset='
CLEANR = re.compile('<.*?>') 

In [10]:
def parse_rbc_news(days=1):
    if days == 0: days = 1
    offset = 0
    ress = []
    while (True):
        response = rq.get(RBC_FINANCE_URL + str(offset))
        offset += 20
        if (response.status_code == 200):
            news_urls = re.findall('https://www.rbc.ru/finances/\d+/\d+/\d+/\w+', response.text)
            for url in news_urls:
                try:
                    res = {}
                    res['url']= url
                    news = rq.get(url).text
                    soup = bs(news, 'html.parser')
                    res['site'] = 'rbc_finances'
                    res['title'] = soup.body.find(class_="article__header__title-in js-slide-title").text.replace("\xa0", " ").replace("\t", " ").strip()
                    
                    def cleanhtml(raw_html):
                        return re.sub(CLEANR, '', raw_html)

                    raw = soup.body.find('div', class_="article__text article__text_free").find_all('p')
                    res['text'] = ''
                    for i in raw:
                        if (str(i).find('span') == -1):
                            res['text'] += cleanhtml(str(i))
                    try:
                        res['description'] = soup.body.find(class_="article__text__overview").text.replace("\xa0", " ").replace("\t", " ")
                    except:
                        if (len(res['text']) > 100):
                            res['description'] = res['text'][:100] + '...'
                        else:
                            res['description'] = res['text']
                    date_raw =re.findall('\d{4}-\d+-\d+T\d+:\d+:\d+' ,str(soup.body.find(class_="article__header__date")))[0]
                    date, time = date_raw.split('T')
                    date = date.split('-')
                    time = time.split(':')
                    res['timestamp'] = dt.datetime(int(date[0]), int(date[1]), int(date[2]), int(time[0]), int(time[1]), int(time[2])).timestamp()
                    res['time'] = str(date_raw).replace("T", " ")
                    if (days * 86400 + res['timestamp'] < dt.datetime.now().timestamp()):
                        return pd.json_normalize(ress)
                    ress.append(res)
                except Exception as e:
                    print(e, url)
        else:
            return pd.json_normalize(ress)

In [11]:
df = pd.read_csv('rbc_finances_new.csv')
df = df.sort_values(by=['time'], ascending=False)
t = (datetime.today().date() - datetime.strptime(df[['time']].values[0][0][:10], '%Y-%m-%d').date()).days
df0 = parse_rbc_news(t)
df0 = df0[['time','url', 'site', 'title', 'text']]
df0['text'] = df0['text'].replace(r'\s+|\\n', ' ', regex=True)
day1 = pd.concat([df, df0], axis=0, ignore_index=True)
day1 = day1.drop_duplicates (subset=['time'], keep='last')
day1 = day1.sort_values(by=['time'], ascending=True)
day1 = day1[day1['time'].map(lambda x: x[0:10]) <= str(yesterday)]
day1.to_csv('rbc_finances_new.csv', index = False)

In [12]:
inter1 = pd.read_csv('rbc_finances_new.csv')
inter1 = inter1.dropna()
inter1['stock'] = ''
for i, j in list(zip(stock,l)):
      inter1['stock'][inter1['text'].str.lower().str.contains(j)] = i

result1 = pd.concat([inter1[inter1['stock'] == 'SBER'], inter1[inter1['stock'] == 'ROSN'], inter1[inter1['stock'] == 'GAZP'], inter1[inter1['stock'] == 'RTKM'], inter1[inter1['stock'] == 'MOEX']], axis=0, ignore_index=True)        
result1

,time,url,site,title,text,stock
0,2023-01-10 00:00:20,https://www.rbc.ru/finances/10/01/2023/63bc010...,rbc_finances,Сбербанк повысил ставки по ипотеке,Сбербанк с 10 января повысил ставки по ипотеке...,SBER
1,2023-01-17 10:16:34,https://www.rbc.ru/finances/17/01/2023/63c648e...,rbc_finances,Чистая прибыль Сбербанка за год составила ₽300...,По итогам 2022 года чистая прибыль Сбербанка п...,SBER
2,2023-01-18 12:00:21,https://www.rbc.ru/finances/18/01/2023/63c7b4a...,rbc_finances,Сбербанк начал работу в Крыму,"Сбербанк начал работу в Крыму, первые офисы от...",SBER
3,2023-01-19 08:00:52,https://www.rbc.ru/finances/19/01/2023/63c7fb8...,rbc_finances,И.о. главы «Сбербанк Капитала» ушел из компании,Исполняющий обязанности генерального директора...,SBER
4,2023-01-25 09:00:24,https://www.rbc.ru/finances/25/01/2023/63cebaf...,rbc_finances,Крупнейший поставщик терминалов для приема кар...,Один из крупнейших производителей терминалов д...,SBER
...,...,...,...,...,...,...
276,2023-10-24 19:00:00,https://www.interfax.ru/business/927388,interfax_business,Рынок акций РФ превысил 1100 пунктов по индекс...,Рынок акций РФ во вторник вечером продемонстри...,MOEX
277,2023-10-25 09:03:36,https://www.rbc.ru/finances/25/10/2023/6537b91...,rbc_finances,На Мосбирже резко выросло число попыток разогн...,Московская биржа в третьем квартале 2023 года ...,MOEX
278,2023-11-03 06:00:25,https://www.rbc.ru/finances/03/11/2023/65439fb...,rbc_finances,«Южуралзолото» перед IPO оценили в 120 млрд руб.,Оценка компании «Южуралзолото» перед IPO соста...,MOEX
279,2023-11-07 13:29:58,https://www.rbc.ru/finances/07/11/2023/654a100...,rbc_finances,На СПБ Бирже сменится руководитель,Первый заместитель гендиректора СПБ Биржи Евге...,MOEX


## Smartlab

In [13]:
def get_date(date):
    month_list = {'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04', 'мая': '05', 'июня': '06',
           'июля': '07', 'августа': '08', 'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'}
    date_list = date.split(' ')
    month = month_list[date_list[1]]
    return date_list[2] +'-' + month + '-'+ date_list[0] + ' ' + date_list[-1] + ':00'

In [14]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [15]:
def parse_smartlab_news(news_n, stock):
    
        ress = []
#         n = rq.get(f'https://smart-lab.ru/forum/news/{stock}')
#         news_n = re.findall(f'/forum/news/{stock}/page\w+', n.text)
#         news_n = int(re.findall('[0-9.]+', news_n[-1])[0])
        if news_n == 0: n = news_n + 2
        else: n = news_n + 1

        for url in range(1, n):
                    try:   
                        
                        business_url = f'https://smart-lab.ru/forum/news/{stock}/page' + str(url)
                        rn = rq.get(business_url)
                        news_urls = re.findall('/blog/\w+.php', rn.text)
                        news_urls = list(map(lambda x: x.split('/')[-1], news_urls))
                        news_urls = list(map(lambda x: 'https://smart-lab.ru/blog/news/' + x, news_urls))

                    except Exception as e:
                                    print(e, url)
                    for u in news_urls:
                                    
                                    try:
                                

                                                res = {}
                                                nv = rq.get(u).content
                                                s = bs(nv, 'html.parser')


                                                res['time'] = s.find_all(class_="date")
                                                res['time'] = re.findall('\d+\s\w+\s\d+, \d+:\d+', str(res['time']))[0].replace(",", "")
                                                res['time'] = get_date(res['time'])

                                                res['url']= u
                                                res['site'] = 'smart-lab.ru'

                                                res['title'] = s.title.text

                                                res['text'] = s.find_all('div', class_='content')[1].text
                                                res['text'] = re.sub(re.compile('[\n\r\n]'),'', res['text'])

                                                ress.append(res)   

                           
                                    except Exception as e:
                                                        print(e, u)

        return pd.json_normalize(ress)

In [16]:
for i in stock:
    d1 = pd.read_csv(f'smartlab_{i}_new.csv')
    d1 = d1.sort_values(by=['time'], ascending=False)
    t = (datetime.today().date() - datetime.strptime(d1[['time']].values[0][0][:10], '%Y-%m-%d').date()).days
    d2 = parse_smartlab_news(t, i)
    d2['text'] = d2['text'].replace(re.compile(emoji_pattern), ' ', regex=True)
    d2['title'] = d2['title'].replace(re.compile(emoji_pattern), ' ', regex=True)
    day2 = pd.concat([d1, d2], axis=0, ignore_index=True)
    day2 = day2.drop_duplicates (subset=['time'], keep='last')
    day2 = day2.sort_values(by=['time'], ascending=True)
    day2 = day2[day2['time'].map(lambda x: x[0:10]) <= str(yesterday)]
    day2.to_csv(f'smartlab_{i}_new.csv', index = False)

### Создаем итоговые отдельные новостные датафреймы по акциям

In [17]:
rbc = pd.read_csv('rbc_finances_new.csv')
inter = pd.read_csv('interfax_business_new.csv')
result = pd.concat([rbc, inter], axis=0, ignore_index=True)
result = result.dropna()
result['stock'] = ''
for i, j in list(zip(stock,l)):
                          df = pd.read_csv(f'smartlab_{i}_new.csv')
                          df['stock'] = i
                          result['stock'][result['text'].str.lower().str.contains(j)] = i
                          ress = pd.concat([result[result['stock'] == i], df], axis=0, ignore_index=True)
                          ress = ress.sort_values(by=['time'], ascending=True)
                          ress.to_csv(f'{i}_news.csv', index = False)